In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [42]:
# loads all original data
df = pd.read_csv('../datasets/correct/usage.csv')
df.head()

,date,usage
0,2021-04-12T00:00:02.404000,570
1,2021-04-12T00:00:12.911000,570
2,2021-04-12T00:00:23.429000,580
3,2021-04-12T00:00:33.935000,600
4,2021-04-12T00:00:44.455000,590


In [43]:
df['date'] = pd.to_datetime(df['date'])
times = df['date'].copy()
times.head()

0   2021-04-12 00:00:02.404
1   2021-04-12 00:00:12.911
2   2021-04-12 00:00:23.429
3   2021-04-12 00:00:33.935
4   2021-04-12 00:00:44.455
Name: date, dtype: datetime64[ns]

In [4]:
def distance(d1, d2):
    return abs((d2-d1).seconds)

In [3]:
for i in range(1, len(times)):
    diff = distance(times[i-1], times[i])
    if diff > 50:
        print(diff, i, times[i-1], times[i])

NameError: name 'times' is not defined

In [45]:
df['day'] = [x.day for x in df['date']]
df['month'] = [x.month for x in df['date']]
df.head()

,date,usage,day,month
0,2021-04-12 00:00:02.404,570,12,4
1,2021-04-12 00:00:12.911,570,12,4
2,2021-04-12 00:00:23.429,580,12,4
3,2021-04-12 00:00:33.935,600,12,4
4,2021-04-12 00:00:44.455,590,12,4


In [50]:
index19 = df[(df['day'] == 19) & (df['month'] == 4)].index
index22 = df[(df['day'] == 22) & (df['month'] == 4)].index
index27 = df[(df['day'] == 27) & (df['month'] == 4)].index
index19

Int64Index([57554, 57555, 57556, 57557, 57558, 57559, 57560, 57561, 57562,
            57563,
            ...
            65603, 65604, 65605, 65606, 65607, 65608, 65609, 65610, 65611,
            65612],
           dtype='int64', length=8059)

In [51]:
df_ng = df.drop(index19, inplace=False)
df_ng.drop(index22, inplace=True)
df_ng.drop(index27, inplace=True)

In [52]:
df_ng.to_csv('../datasets/processed/removed_gabs.csv', index=False)

In [5]:
# Get the processed file
dfg = pd.read_csv('../datasets/processed/removed_gabs.csv')

In [6]:
dfg['date'] = pd.to_datetime(dfg['date'])

In [7]:
def skipped(diff):
    return int(round(diff/10.0) - 1)

In [10]:
gab_time_index = []
gab_size = []
times = dfg['date'].copy()
for i in range(1, len(times)):
    diff = distance(times[i-1], times[i])
    if skipped(diff) > 0:
        gab_time_index.append(i)
        gab_size.append(skipped(diff))

In [11]:
dfg.head()

,date,usage,day,month
0,2021-04-12 00:00:02.404,570,12,4
1,2021-04-12 00:00:12.911,570,12,4
2,2021-04-12 00:00:23.429,580,12,4
3,2021-04-12 00:00:33.935,600,12,4
4,2021-04-12 00:00:44.455,590,12,4


In [16]:
usage_series = dfg['usage'].copy()

In [17]:
dfc = dfg.copy()
time_line = []
usage_line = []
offset = 0
for g in range(0, len(gab_size)):
    time_line.clear()
    usage_line.clear()
    for i in range(0, gab_size[g]):
        usage = (usage_series[gab_time_index[g]-1] + usage_series[gab_time_index[g]]) / 2
        time = dfg['date'][gab_time_index[g]-1] + (timedelta(seconds=10) * (i+1))
        print(usage, time)
        time_line.append(time)
        usage_line.append(usage)

        
    df = pd.DataFrame({"date": time_line, "usage": usage_line})
    dfc = pd.concat([dfc.iloc[:gab_time_index[g]+offset], df.iloc[:], dfc.iloc[gab_time_index[g]+offset:]])
    offset += len(df)
dfc.reset_index(drop=True)
dfc.head()

870.0 2021-04-12 12:08:30.205000
1600.0 2021-04-13 07:19:26.017000
1645.0 2021-04-13 14:04:35.711000
3210.0 2021-04-14 23:12:04.086000
-340.0 2021-04-15 10:19:26.386000
-1690.0 2021-04-15 12:08:40.538000
2025.0 2021-04-15 20:00:04.332000
-710.0 2021-04-16 12:08:41.232000
-530.0 2021-04-16 15:16:14.617000
-395.0 2021-04-16 17:09:21.716000
-1810.0 2021-04-17 13:19:26.401000
-1100.0 2021-04-18 12:08:50.888000
-115.0 2021-04-18 13:08:13.116000
860.0 2021-04-19 00:00:01.867000
-105.0 2021-04-20 12:08:55.304000
645.0 2021-04-20 17:04:11.535000
3695.0 2021-04-21 19:19:21.711000
1380.0 2021-04-23 12:09:07.202000
1920.0 2021-04-23 19:31:51.504000
1920.0 2021-04-23 19:32:01.504000
1790.0 2021-04-23 19:36:03.754000
1190.0 2021-04-23 21:53:43.243000
1245.0 2021-04-23 22:09:10.524000
990.0 2021-04-23 22:19:17.677000
1255.0 2021-04-23 22:35:07.261000
1225.0 2021-04-24 08:25:43.495000
-270.0 2021-04-24 09:17:33.829000
320.0 2021-04-24 10:33:33.702000
320.0 2021-04-24 10:33:43.702000
1890.0 2021-04-24

,date,usage,day,month
0,2021-04-12 00:00:02.404,570.0,12.0,4.0
1,2021-04-12 00:00:12.911,570.0,12.0,4.0
2,2021-04-12 00:00:23.429,580.0,12.0,4.0
3,2021-04-12 00:00:33.935,600.0,12.0,4.0
4,2021-04-12 00:00:44.455,590.0,12.0,4.0


AttributeError: 'Series' object has no attribute 'seconds'

In [18]:
dfc.to_csv('../datasets/processed/filled_gabs.csv', index=False)